# Criação de Meta-Base

## Considerações sobre Aprendizado Ativo

Para que os experimentos sejam executados, é necessária a criação de uma meta-base a respeito do processo de **aprendizado ativo** (AL).  Contudo, algumas variáveis podem influenciar diretamente o processo de aprendizado ativo, sendo elas:

- Quantidade Inicial de Dados Rotulados (  $\vert X^0_{labeled}\vert$ )
- Quantidade de consultas a serem realizados ($q$)
- Estratégia de consulta utilizada (*query strategy*)
- Quantidade de instâncias retornadas por uma consulta (*batch size*)
- Método utilizado para ordenar as instâncias retornadas por uma consulta
- Tamanho do conjunto de teste para avaliar o aprendizado


### Quantidade Inicial de Dados Rotulados

Em tese, a ideia principal do AL é tentar popuar esforços de anotação ao rotular as instâncias que melhor descrevem um conjunto de dados. Dessa forma, na maioria dos casos lidamos com situações em que há uma pequena quantidade de instâncias rotuladas, apenas o suficiente para iniciar o processo de aprenzidado ativo

> Qual a quantidade ideal de dados anotados inicialmente? 

Como o nosso objetivo é simular várias configurações diferentes de active learning para cada uma das bases, é necessário pensar em alguma manera de deixar esse processo o mais reprodutível possível, algumas ideias são: 

1.  $\vert X_{labeled} \vert = n$, onde $n$ representa uma constante arbitrária
2.  $\vert X_{labeled} \vert = c$, onde $c$ representa o número de classes no problema de classificação
2.  $\vert X_{labeled} \vert = \alpha\times\vert X\vert$, onde $\alpha \in (0, 1]$ representa uma constante de proporcionalidade

### Quantidade de consultas a serem realizadas

Como a quantidade de instâncias varia conforme o conjunto de dados, também devemos decidir como escolheremos esse parâmetro. Dessa forma podemos utilizar soluções similares às citadas acima, porém baseado no tamanho do conjunto de dados

### Estratégia de Consulta

Estratégias baseadas na incerteza de um classificador:
- [ ] Classification Uncertainty 
- [ ] Classification Margin
- [ ] Classification Entropy


Estratégias baseadas em discordância entre modelos

- [ ] Vote Entropy
- [ ] Consensus Entropy
- [ ] Max Disagreement

### Quantidade de instâncias retornadas por consulta

Originalmente, as estratégias de consulta está preocupadas em selecionar a instância mais interessante para ser rotulado, contudo o que ocorre geralmente é que o anotador não está disposto a anotar apenas uma instância e esperar uma próxima iteração do processo de AL ser executado novamente. Muito tempo e recursos podem ser otimizados se mais de uma instância for retornada por consulta.

Todavia, simplesmente retornar as $n$ consultas com a maior pontuação, pode não ser uma boa ideia (embora as vezes seja uma alternativa). Sendo assim Cardoso et. al formularam uma maneira de ranquear essas instâncias

$score = \alpha(1 - \Phi(x, X_{labeled})) + (1 - \alpha) U(x),$ 

Onde $\alpha = \frac{|X_{unlabeled}|}{|X_{unlabeled}| + |X_{labeled}|}$, $U(x)$ é a incerteza das predições para $x$ e $\Phi$ é uma função de similaridade. Dessa forma é possível medir o quão bem o espaço de características foi explorado perto de $x$

## Criação de Meta-base

### Simulação de Cenário

In [1]:
from functools import partial
from typing import Union
import warnings

from modAL.models import ActiveLearner, Committee
from sklearn import metrics
from sklearn.compose import ColumnTransformer
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from tqdm.notebook import tqdm
import numpy as np
import openml
import pandas as pd

# Query Strategies
from modAL.uncertainty import uncertainty_sampling, margin_sampling, entropy_sampling
from modAL.disagreement import vote_entropy_sampling, consensus_entropy_sampling, max_disagreement_sampling
from modAL.batch import uncertainty_batch_sampling


class MetaBaseBuilder:
    uncertainty_strategies = [
        uncertainty_sampling,
        uncertainty_batch_sampling,
        margin_sampling,
        entropy_sampling,
    ]
    disagreement_strategies = [
        vote_entropy_sampling,
        consensus_entropy_sampling,
        max_disagreement_sampling
    ]
    
    def __init__(self, estimators,
                 query_strategies,
                 n_queries=50,
                 batch_size=5,
                 committee_size=3,
                 initial_l_size=-1, ):
        self.__estimators = estimators
        self.__query_strategies = query_strategies
        self.__n_queries = n_queries
        self.__batch_size = batch_size
        self.__committee_size = committee_size
        self.__initial_l_size = initial_l_size

    def build(self, dataset: openml.datasets.OpenMLDataset):
        # fazer tratamento de dados
        X, y = self.__load_data(dataset)

        self.classes_ = np.unique(y)
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

        # Faz com que inicialmente haja uma instância rotulada por classe
        labeled_index = [np.random.choice(np.where(y_train == cls)[0])
                         for cls in np.unique(y_train)]

        if len(labeled_index) < self.__initial_l_size:
            additional_index = np.random.choice(len(y_train),
                                                    size=self.__initial_l_size,
                                                    replace=False)
            labeled_index.extend(additional_index.tolist())

        l_X_pool = X_train[labeled_index]
        l_y_pool = y_train[labeled_index]

        u_X_pool = np.delete(X_train, labeled_index, axis=0)
        u_y_pool = np.delete(y_train, labeled_index, axis=0)

        with warnings.catch_warnings():
            
            warnings.filterwarnings("ignore", category=ConvergenceWarning)
            
            learners = self.__build_learners(l_X_pool, l_y_pool)

            for learner in learners:
                self.__teach_learner(learner, u_X_pool, u_y_pool,
                                     X_test, y_test)

    def __load_data(self, dataset: openml.datasets.OpenMLDataset):
        X, y, categorical_indicator, _ = dataset.get_data(
            target=dataset.default_target_attribute)

        encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
        transformers = [('one-hot-encoder', encoder, categorical_indicator)]

        preprocessor = ColumnTransformer(transformers, remainder='passthrough')

        le = LabelEncoder()

        X, y = preprocessor.fit_transform(X), le.fit_transform(y)

        return X, y

    def __build_learners(self, X_training, y_training):
        return [self.__build_learner(estimator, strategy,
                                     X_training, y_training)
                for estimator in self.__estimators
                for strategy in self.__query_strategies]

    def __build_learner(self, estimator, strategy,
                        X_training, y_training):

        query_strategy = partial(strategy,
                                 n_instances=self.__batch_size)

        # Copia o nome para facilitar debbug
        query_strategy.__name__ = strategy.__name__

        args = {
            'estimator': estimator,
            'query_strategy': query_strategy,
            'X_training': X_training,
            'y_training': y_training
        }

        if strategy in self.uncertainty_strategies:
            return ActiveLearner(**args)
        else:
            learner_list = list()

            args.pop('query_strategy')
            for _ in range(self.__committee_size):
                learner = ActiveLearner(**args)
                learner_list.append(learner)

            committee = Committee(learner_list=learner_list,
                                  query_strategy=query_strategy)
            return committee

    def __teach_learner(self, learner: Union[ActiveLearner, Committee], X_pool, y_pool,
                        X_test, y_test):

        scores = self.__eval_learner(learner, X_test, y_test)

        pbar = tqdm(range(self.__n_queries), leave=False)
        estimator = (learner.estimator if isinstance(learner, ActiveLearner)
                     else learner.learner_list[0].estimator)

        pbar.set_description(
            desc=(f'model: {type(estimator).__name__}, '
                  f'strategy: {learner.query_strategy.__name__}').rjust(70),
            refresh=True)

        pbar.set_postfix(
            U=X_pool.shape[0],
            L=(learner.X_training.shape[0] if not isinstance(learner, Committee)
               else learner.learner_list[0].X_training.shape[0]),
            **scores,
            refresh=True)

        for idx in pbar:

            query_index, query_instance = learner.query(X_pool)
            
            learner.teach(X=X_pool[query_index], y=y_pool[query_index])

            X_pool = np.delete(X_pool, query_index, axis=0)
            y_pool = np.delete(y_pool, query_index, axis=0)

            model_score = learner.score(X_test, y_test)

            pbar.set_postfix(
                U=X_pool.shape[0],
                L=(learner.X_training.shape[0] if not isinstance(learner, Committee)
                   else learner.learner_list[0].X_training.shape[0]),
                accuracy=model_score,
                refresh=True)
            
        pbar.close()
        
    def __eval_learner(self, learner: ActiveLearner, X, y_true):
        y_pred = learner.predict(X)
        scores = dict()
        scores['accuracy'] = metrics.accuracy_score(y_pred, y_true)

        return scores

#### Classificadores

In [10]:
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

class SVCLinear(SVC):
    pass
    
clf_list = [SVCLinear(kernel='linear', probability=True),
            SVC(probability=True),
            RandomForestClassifier(),
            KNeighborsClassifier(),
            MLPClassifier(),
            LogisticRegression(),
            DecisionTreeClassifier(),
            GaussianNB(),
]


#### Estratégias de consulta

In [3]:
query_strategies = [ 
    uncertainty_sampling,
    uncertainty_batch_sampling, 
    margin_sampling,
    entropy_sampling,
    consensus_entropy_sampling,
    vote_entropy_sampling,
    max_disagreement_sampling
]

### Conjuntos de dados

In [4]:
from config import dataset_ids

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    datasets = openml.datasets.get_datasets(dataset_ids)

len(datasets)

70

#### Teste

In [11]:
# %debug 
import multiprocessing

test_builder = MetaBaseBuilder(estimators=clf_list,
                               query_strategies=query_strategies,
                               n_queries=5,
                               initial_l_size=5)


test_builder.build(datasets[0])

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]